# resultm

In [ ]:
open rust
open rust_operators

In [ ]:
//// test

open testing

## resultm

### from_option_error

In [ ]:
inl from_option_error error opt =
    match opt with
    | Some x => Ok x
    | None => Error error

### from_option

In [ ]:
inl from_option opt =
    opt |> from_option_error "resultm.from_option / Option does not have a value."

### flatten_option

In [ ]:
inl flatten_option forall t u. (x : option (result (option t) u)) : result (option t) u =
    match x with
    | Some (Error x) => Error x
    | Some (Ok (Some x)) => Ok (Some x)
    | _ => Ok None

### flatten

In [ ]:
inl flatten forall t u. (x : result (result t u) u) : result t u =
    match x with
    | Ok x => x
    | Error x => Error x

### get

In [ ]:
inl get forall t e. (source : result t e) : t =
    match source with
    | Ok x => x
    | Error x =>
        backend_switch {
            Fsharp = fun () => $'$"resultm.get / Result value was Error: {!x}"' : string
            Python = fun () => $'f"resultm.get / Result value was Error: {!x}"' : string
        }
        |> failwith

### map

In [ ]:
inl map forall t e u. (fn : t -> u) (source : result t e) : result u e =
    match source with
    | Ok x => x |> fn |> Ok
    | Error x => Error x

### map_error

In [ ]:
inl map_error forall t e u. (fn : e -> u) (source : result t e) : result t u =
    match source with
    | Ok x => Ok x
    | Error x => x |> fn |> Error

### unwrap_err

In [ ]:
inl unwrap_err forall t u. (x : result t u) : u =
    match x with
    | Ok x =>
        backend_switch {
            Fsharp = fun () => $'$"resultm.unwrap_err / x: {!x}"' : string
            Python = fun () => $'f"resultm.unwrap_err / x: {!x}"' : string
        }
        |> failwith
    | Error x => x

### ok

In [ ]:
inl ok forall t. (x : result t _) : option t =
    match x with
    | Ok x => Some x
    | Error _ => None

## fsharp

### result'

In [ ]:
nominal result' t u = $'Result<`t, `u>'

### unbox

In [ ]:
inl unbox forall t u. (x : result' t u) : result t u =
    inl ok x : result t u = Ok x
    inl error x : result t u = Error x
    inl ok = join ok
    inl error = join error
    real
        typecase t with
        | () => $'match !x with Ok () -> !ok () | Error x -> !error x' : result t u
        | _ => $'match !x with Ok x -> !ok x | Error x -> !error x' : result t u

### box

In [ ]:
inl box forall t u. (x : result t u) : result' t u =
    match x with
    | Ok x => $'Ok !x '
    | Error err => $'Error !err '

## rust

### anyhow_result

In [ ]:
nominal anyhow_result t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"anyhow::Result<$0>\")>]\n#endif\ntype anyhow_Result<'T> = class end"
        $'' : $'anyhow_Result<`t>'
    )

### anyhow_error

In [ ]:
nominal anyhow_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"anyhow::Error\")>]\n#endif\ntype anyhow_Error = class end"
        $'' : $'anyhow_Error'
    )

inl anyhow_error error =
    !\\(error, $'"anyhow::anyhow\!($0)"')

### try'

In [ ]:
inl try' forall t u. (x : result' t u) : t =
    inl is_unit =
        real
            typecase t with
            | () => true
            | _ => false
    if is_unit
    then (!\\(x, $'"true; $0?"') : bool) |> fun _ => $''
    else !\\(x, $'"$0?"')

### to_try

In [ ]:
inl to_try forall t u. (x : result' t u) : rust.try t =
    !\\(x, $'"$0"')

### unwrap'

In [ ]:
inl unwrap' forall t u. (x : result' t u) : t =
    run_target function
        | Rust _ => fun () => !\\(x, $'"$0.unwrap()"')
        | _ => fun () => $'match !x with Ok x -> x | Error e -> failwith $"resultm.unwrap\' / e: {e}"'

### unwrap_err'

In [ ]:
inl unwrap_err' forall t u. (x : result' t u) : u =
    $'match !x with Ok x -> failwith $"resultm.unwrap_err\' / x: %A{x}" | Error x -> x'

### unbox'

In [ ]:
inl unbox' forall t u. (x : result' t u) : result t u =
    inl ok x : result t u = Ok x
    inl ok = join ok
    inl error x : result t u = Error x
    inl error = join error
    real
        typecase t with
        | () =>
            (~!\\)
                `((result' t u -> result t u) * (result' t u -> result t u))
                `(result t u)
                ((ok, error, x), ($'"match $2 { Ok(()) => $0(()), Err(e) => $1(e) }"' : string))
        | _ =>
            (~!\\)
                `((result' t u -> result t u) * (result' t u -> result t u))
                `(result t u)
                ((ok, error, x), ($'"match $2 { Ok(x) => $0(x), Err(e) => $1(e) }"' : string))

### map'

In [ ]:
inl map' forall t e u. (fn : t -> u) (source : result' t e) : result' u e =
    (!\\(source, $'"true; let _result_map_ = $0.map(|x| { //"') : bool) |> ignore
    (!\\(fn !\($'"x"'), $'"true; $0 })"') : bool) |> ignore
    !\($'"_result_map_"')

### map''

In [ ]:
inl map'' forall t e u. (fn : t -> u) (source : result' t e) : result' u e =
    inl fn = join fn
    inl source = join source
    !\($'"!source.map(|x| !fn(x))"')

### map_error'

In [ ]:
inl map_error' forall t e u. (fn : e -> u) (source : result' t e) : result' t u =
    inl fn = join fn
    run_target_args (fun () => fn) function
        | Rust _ => fun fn =>
            !\\((source, fn), $'"$0.map_err(|x| $1(x))"')
        | _ => fun fn =>
            $'match !source with Ok x -> Ok x | Error x -> Error (!fn x)' : result' t u

### map_error''

In [ ]:
inl map_error'' forall t e u. (fn : e -> u) (source : result' t e) : result' t u =
    (!\\(source, $'"true; let _result_map_error__ = $0.map_err(|x| { //"') : bool) |> ignore
    (!\\(fn !\($'"x"'), $'"true; $0 })"') : bool) |> ignore
    !\($'"_result_map_error__"')

### option_ok_or

In [ ]:
inl option_ok_or forall t e. (e : e) (source : optionm'.option' t) : result' t e =
    !\\(source, $'"$0.ok_or(!e)"')

### unwrap_or_else

In [ ]:
inl unwrap_or_else forall t e u. (fn : e -> u) (source : result' t e) : u =
    (!\\(source, $'"true; let _result_unwrap_or_else = $0.unwrap_or_else(|x| { //"') : bool) |> ignore
    (!\\(fn !\($'"x"'), $'"true; $0 })"') : bool) |> ignore
    !\($'"_result_unwrap_or_else"')

### map_or_else

In [ ]:
inl map_or_else forall t e u v. (fn : e -> v) (fn2 : u -> v) (source : result' t e) : v =
    (!\\(source, $'"true; let _result_map_or_else = $0.map_or_else(|x| { //"') : bool) |> ignore
    (!\\(fn !\($'"x"'), $'"true; $0 }, |x| { //"') : bool) |> ignore
    (!\\(fn2 !\($'"x"'), $'"true; $0 })"') : bool) |> ignore
    !\($'"_result_map_or_else"')

### as_ref

In [ ]:
inl as_ref forall t e. (source : result' t e) : result' (rust.ref t) (rust.ref e) =
    !\($'"!source.as_ref()"')

### as_ref'

In [ ]:
inl as_ref' forall t e. (source : rust.ref (result' t e)) : result' (rust.ref t) (rust.ref e) =
    !\($'"!source.as_ref()"')

### unwrap_or'

In [ ]:
inl unwrap_or' forall t u. (default : t) (x : result' t u) : t =
    !\\((x, default), $'"$0.unwrap_or($1)"')

### expect

In [ ]:
inl expect forall t u. (error : rust.ref string) (x : result' t u) : t =
    !\($'"!x.expect(&!error)"')

### is_err

In [ ]:
inl is_err forall t u. (x : result' t u) : bool =
    run_target function
        | Rust _ => fun () => !\\(x, $'"$0.is_err()"')
        | _ => fun () => true

### ok'

In [ ]:
inl ok' forall t. (x : result' t _) : optionm'.option' t =
    run_target function
        | Rust _ => fun () => !\\(x, $'"$0.ok()"')
        | _ => fun () => $'match !x with Ok x -> Some x | Error _ -> None'

### err

In [ ]:
inl err forall t u. (x : u) : result' t u =
    run_target function
        | Rust _ => fun () => !\\(x, $'"Err($0)"')
        | _ => fun () => $'!x |> Error'

### ok''

In [ ]:
inl ok'' forall t u. (x : t) : result' t u =
    run_target function
        | Rust _ => fun () => !\\(x, $'"Ok($0)"')
        | _ => fun () => $'!x |> Ok'

### transpose

In [ ]:
inl transpose forall t u. (x : optionm'.option' (result' t u)) : result' (optionm'.option' t) u =
    !\\(x, $'"$0.transpose()"')

### rc_try_unwrap

In [ ]:
inl rc_try_unwrap forall t. (x : rust.rc t) : result' t (rust.rc t) =
    !\\(x, $'"std::rc::Rc::try_unwrap($0)"')

In [ ]:
//// test
///! rust

rust.new_rc true
|> rc_try_unwrap
|> unbox
|> _assert_eq (Ok true)

__assert_eq / actual: US0_0(true) / expected: US0_0(true)
